# Chile Consolidación de Datos territoriales

In [ ]:
import pandas as pd
import json

## Extración

### Listado y Nombre de Regiones según la Ley 21.074 del 2018

In [ ]:
ley_regiones_df = pd.read_csv("sources/ley-regiones.csv", dtype=str)
ley_regiones_df

### Consolidado de Regiones, Provincias y Comunas hasta el 2018

In [ ]:
observatoriologistico_df = pd.read_csv("sources/observatoriologistico.csv", dtype=str)
observatoriologistico_df

### Estandarización Internacional Territorial Chile

In [ ]:

iso_df = pd.read_csv("sources/iso.csv")
iso_df

## Transformaciones

In [ ]:
def apply_replacements(text, replacements):
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

replacements = {
    "Aisén del General Carlos Ibañez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Región Metropolitana de Santiago": "Metropolitana de Santiago",
}
iso_df["corrected_name"] = iso_df["Subdivision name"].apply(
    lambda x: apply_replacements(x, replacements)
)

# Preprocesar iso_df para hacer la correspondencia más directa y limpiar el código ISO
iso_df["clean_iso_code"] = iso_df["3166-2 code"].str.replace("*", "")

# Unir los dataframes en los nombres corregidos y necesarios
merged_df = observatoriologistico_df.merge(
    iso_df[["corrected_name", "clean_iso_code"]],
    left_on="Nombre Región",
    right_on="corrected_name",
    how="left",
)
merged_df = merged_df.merge(
    ley_regiones_df[["código", "nombre"]],
    left_on="Código Región",
    right_on="código",
    how="left",
)
merged_df

## Carga

### CSV
#### Construcción de Regiones CSV

In [ ]:
# Seleccionar y renombrar las columnas necesarias para el DataFrame final
regions_df = merged_df[
    ["Código Región", "nombre", "Nombre Región", "Abreviatura Región", "clean_iso_code"]
]
regions_df.columns = ["id", "name", "short_name", "abbreviation", "iso_code"]
# Pylance:
regions_df = regions_df.drop_duplicates(subset=["id"], keep="last")

# Guardar a CSV y mostrar el DataFrame
regions_df.to_csv("csv/regions.csv", index=False)
regions_df

#### Construcción de Provincias CSV

In [ ]:
provinces_df = observatoriologistico_df[
    ["Código Provincia", "Nombre Provincia", "Código Región"]
].copy()
provinces_df.columns = ["id", "name", "region_id"]

provinces_df = provinces_df.drop_duplicates(subset=["id"], keep="last")

provinces_df.to_csv("csv/provinces.csv", index=False)
provinces_df

#### Construcción de Comunas CSV

In [ ]:
communes_df = observatoriologistico_df[
    ["Código Comuna 2018", "Nombre Comuna", "Código Provincia"]
].copy()
communes_df.columns = ["id", "name", "province_id"]

communes_df.to_csv("csv/communes.csv", index=False)
communes_df

## JSON

### Datos Territoriales como objeto JSON

In [ ]:
# Agrupar comunas por provincia_id y construir el diccionario de comunas
communes_grouped = communes_df.groupby("province_id")
provinces_df["communes"] = provinces_df["id"].apply(
    lambda x: {
        row["id"]: {"id": row["id"], "name": row["name"]}
        for _, row in communes_grouped.get_group(x).iterrows()
        if x in communes_grouped.groups
    }
)

# Agrupar provincias por region_id y construir el diccionario de provincias
provinces_grouped = provinces_df.groupby("region_id")
regions_df["provinces"] = regions_df["id"].apply(
    lambda x: {
        row["id"]: {"id": row["id"], "name": row["name"], "communes": row["communes"]}
        for _, row in provinces_grouped.get_group(x).iterrows()
        if x in provinces_grouped.groups
    }
)

# Construir el diccionario final para el país, asumiendo que el país es Chile
cl_country_object = {
    "country": "Chile",
    "countryCode": "CL",
    "regions": {
        row["id"]: {
            "id": row["id"],
            "name": row["name"],
            "shortName": row["short_name"],
            "abbreviation": row["abbreviation"],
            "isoCode": row["iso_code"],
            "provinces": row["provinces"],
        }
        for _, row in regions_df.iterrows()
    },
}

# Convertir el diccionario a JSON
json_object = json.dumps(cl_country_object, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open("json/object.json", "w", encoding="utf-8") as f:
    f.write(json_object)

# Imprimir el JSON para verificar
print(json_object)

### Datos Territoriales como arreglo JSON

In [ ]:
regions =  [
    {
        "id": row["id"],
        "name": row["name"],
        "shortName": row["short_name"],
        "abbreviation": row["abbreviation"],
        "isoCode": row["iso_code"],
    }
    for _, row in regions_df.iterrows()
]
# Convertir el arreglo a JSON
regions_json = json.dumps(regions, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open("json/regions.json", "w", encoding="utf-8") as f:
    f.write(regions_json)

# Imprimir el JSON para verificar
print(regions)

In [ ]:
provinces = [
    {
        "id": row["id"],
        "name": row["name"],
        "regionId": row["region_id"],
    }
    for _, row in provinces_df.iterrows()
]

# Convertir el arreglo a JSON
provinces_json = json.dumps(provinces, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open("json/provinces.json", "w", encoding="utf-8") as f:
    f.write(provinces_json)

In [ ]:
communes = [
    {
        "id": row["id"],
        "name": row["name"],
        "provinceId": row["province_id"],
    }
    for _, row in communes_df.iterrows()
]

# Convertir el arreglo a JSON
communes_json = json.dumps(communes, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open("json/communes.json", "w", encoding="utf-8") as f:
    f.write(communes_json)

## SQL
### Construcción de Región SQL

In [ ]:
# Definir el modelo de la tabla
table_model = """
CREATE TABLE region (
    id VARCHAR(2) PRIMARY KEY,
    name VARCHAR(255),
    short_name VARCHAR(255),
    abbreviation VARCHAR(10),
    iso_code VARCHAR(10)
);
"""

# Generar las sentencias INSERT
insert_statements = ""
for _, row in regions_df.iterrows():
    insert_statements += f"INSERT INTO region (id, name, short_name, abbreviation, iso_code) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['short_name'].replace("'","''")}', '{row['abbreviation']}', '{row['iso_code']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
sql_script = table_model + "\n" + insert_statements

# Guardar el script SQL en un archivo
with open("sql/regions.sql", "w", encoding="utf-8") as f:
    f.write(sql_script)

# Imprimir el script SQL para verificar
print(sql_script)

### Construcción de Provincia SQL

In [ ]:
# Definir el modelo de la tabla
provinces_table_model = """
CREATE TABLE province (
    id VARCHAR(3) PRIMARY KEY,
    name VARCHAR(255),
    region_id VARCHAR(2),
    FOREIGN KEY (region_id) REFERENCES region(id)
);
"""

# Generar las sentencias INSERT
provinces_insert_statements = ""
for _, row in provinces_df.iterrows():
    provinces_insert_statements += f"INSERT INTO province (id, name, region_id) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['region_id']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
provinces_sql_script = provinces_table_model + "\n" + provinces_insert_statements

# Guardar el script SQL en un archivo
with open("sql/provinces.sql", "w", encoding="utf-8") as f:
    f.write(provinces_sql_script)

# Imprimir el script SQL para verificar
print(provinces_sql_script)

### Construcción de Comuna SQL

In [ ]:
# Definir el modelo de la tabla
communes_table_model = """
CREATE TABLE commune (
    id VARCHAR(5) PRIMARY KEY,
    name VARCHAR(255),
    province_id VARCHAR(3),
    FOREIGN KEY (province_id) REFERENCES province(id)
);
"""

# Generar las sentencias INSERT
communes_insert_statements = ""
for _, row in communes_df.iterrows():
    communes_insert_statements += f"INSERT INTO commune (id, name, province_id) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['province_id']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
communes_sql_script = communes_table_model + "\n" + communes_insert_statements

# Guardar el script SQL en un archivo
with open("sql/communes.sql", "w", encoding="utf-8") as f:
    f.write(communes_sql_script)

# Imprimir el script SQL para verificar
print(communes_sql_script)